In [61]:
import torch
import numpy as np
from sklearn.model_selection import train_test_split
import torch.nn.functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# parameters
training_epochs = 500
batch_size = 16

#데이터 삽입
train = np.loadtxt("data/training.dat")
train=torch.FloatTensor(train)

train_X = train
train_Y = []
for i in range(len(train_X)):
    if i >= 0 and i < 25:
        train_Y.append(0)
    elif i >= 25 and i<50:
        train_Y.append(1)
    else:
        train_Y.append(2)
train_Y = torch.tensor(train_Y)
train_Y = F.one_hot(train_Y)#OneHotEncoding
train_Y = train_Y.type(torch.FloatTensor)


test = np.loadtxt("data/testing.dat")
test = torch.FloatTensor(test)

test_x = test
test_y = []
for i in range(len(test_x)):
    if i < 25:
        test_y.append(0)
    elif i >= 25 and i<50:
        test_y.append(1)
    else:
        test_y.append(2)
test_y = torch.tensor(test_y)
test_y = F.one_hot(test_y)#OneHotEncoding
test_y = test_y.type(torch.FloatTensor)

Train_Data=[]
for i in range(len(train)):
    Train_Data.append((train_X[i],train_Y[i]))

print(Train_Data[0])

(tensor([6.3000, 3.3000, 6.0000, 2.5000]), tensor([1., 0., 0.]))


In [62]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=Train_Data,
                                          batch_size=batch_size, 
                                          shuffle=False, 
                                          drop_last=False)

class NeuralNet(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size): #input: sample의 size  hidden: output의 size
        super(NeuralNet, self).__init__()
        self.input_layer  = torch.nn.Linear(input_size, hidden_size)
        self.hidden_layer1 = torch.nn.Linear(hidden_size, hidden_size)
        self.hidden_layer2 = torch.nn.Linear(hidden_size, hidden_size)
        self.output_layer = torch.nn.Linear(hidden_size, output_size)
        self.batchnorm = torch.nn.BatchNorm1d(hidden_size)
        self.dropout = torch.nn.Dropout(0.2)
        self.relu = torch.nn.ReLU()
        self.soft = torch.nn.Softmax(dim=1)

    def forward(self, x):        
        output = self.input_layer(x)
        output = self.relu(self.hidden_layer1(output))
        output = self.relu(self.hidden_layer2(output))
        output = self.output_layer(output)
        output = self.soft(output)
        return output

In [63]:
#model = NeuralNet(4,16,16,16,3)#입력 열의 개수, 히든 레이어 노드수(2의 n승, 입력받는 열의 개수 보다 많이)
model = NeuralNet(4,32,3)
learning_rate=0.001 #학습율 설정(수정해서 돌려보기) 러닝레이트 스케줄러
criterion=torch.nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)

In [64]:
dtype=torch.long
plt_test=[]
train_Sm = torch.nn.Softmax(dim=1)
for epoch in range(training_epochs):
    avg_cost = 0
    count = 0
    list_train_x=[]
    list_train_y=[]
    total_batch = len(data_loader)

    for X,label in data_loader:
        optimizer.zero_grad()
        train_output=model(X)
        cost = criterion(train_output, label)
        cost.backward()
        optimizer.step()
        
        arg_train_x = torch.argmax(train_output,1)
        arg_train_y = torch.argmax(label,1)
   
        list_train_x += arg_train_x.tolist()
        list_train_y += arg_train_y.tolist()
        avg_cost += cost / total_batch 

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost),end=" ")
    for i in range(len(list_train_x)): #예측 값과 정답 값이 같은 것 count하기
        if list_train_x[i] == list_train_y[i]:
            count+=1
    print('Accuracy:','{:.9f}'.format(count/len(train_X)*100))#예측 값과 정답 값이 같은 지를 퍼센테이지로 나타내기 

Epoch: 0001 cost = 1.103562474 Accuracy: 33.333333333
Epoch: 0002 cost = 1.087666750 Accuracy: 33.333333333
Epoch: 0003 cost = 1.080022573 Accuracy: 33.333333333
Epoch: 0004 cost = 1.072286248 Accuracy: 34.666666667
Epoch: 0005 cost = 1.064560413 Accuracy: 60.000000000
Epoch: 0006 cost = 1.056403995 Accuracy: 66.666666667
Epoch: 0007 cost = 1.047071099 Accuracy: 66.666666667
Epoch: 0008 cost = 1.036294937 Accuracy: 66.666666667
Epoch: 0009 cost = 1.024026752 Accuracy: 66.666666667
Epoch: 0010 cost = 1.010146022 Accuracy: 66.666666667
Epoch: 0011 cost = 0.994554043 Accuracy: 66.666666667
Epoch: 0012 cost = 0.977226913 Accuracy: 66.666666667
Epoch: 0013 cost = 0.958322406 Accuracy: 66.666666667
Epoch: 0014 cost = 0.938596487 Accuracy: 66.666666667
Epoch: 0015 cost = 0.918908298 Accuracy: 66.666666667
Epoch: 0016 cost = 0.899741173 Accuracy: 66.666666667
Epoch: 0017 cost = 0.881964147 Accuracy: 66.666666667
Epoch: 0018 cost = 0.866059065 Accuracy: 66.666666667
Epoch: 0019 cost = 0.8520265

In [65]:
with torch.no_grad():
    model.eval()
    test_count = 0
    test_avg_cost = 0
    list_test_x=[]
    list_test_y=[]
    correct = 0
        
    prediction = model(test_x)
    correct_cost = criterion(prediction, test_y)

    arg_test_x = torch.argmax(prediction,1)
    print(arg_test_x)
    arg_test_y = torch.argmax(test_y,1)
    print(arg_test_y)

    list_test_x += arg_test_x.tolist()
    list_test_y += arg_test_y.tolist()
    
    test_avg_cost += correct_cost / total_batch 
    
    

    for i in range(len(list_test_y)): #예측 값과 정답 값이 같은 것 count하기
        if list_test_x[i] == list_test_y[i]:
            test_count+=1
 
    print('test accuracy:','{:.9f}'.format(test_count/len(test_x)*100))#예측 값과 정답 값이 같은 지를 퍼센테이지로 나타내기 

#     correct_test = [list_test_x[i] == list_test_y[i] for i in range(len(list_test_y))]           
#     for i in range(len(correct_test)): #예측 값과 정답 값이 같은 것 count하기
#         if correct_test[i] == True:
#             test_count+=1
#     print('Accuracy:','{:.9f}'.format(test_count/len(test_x)*100))#예측 값과 정답 값이 같은 지를 퍼센테이지로 나타내기
#     print('result equal: ', [i for i in range(len(correct_test)) if correct_test[i] == False],'\n')#예측 값과 정답 값이 같은 거 출력    


tensor([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2])
test accuracy: 93.333333333
